In [2]:
!pip3 install duckdb

In [115]:
import duckdb
from pathlib import Path

In [116]:

# db = Path("data/duckdb/claims_data_preprocessed.duckdb")
# db = Path("D:/Projekte/202305_Synthetic_data/cuong/eval-synth-data-privacy/data/duckdb/claims_data_preprocessed.duckdb")
db = Path("D:/Projekte/202305_Synthetic_data/cuong/eval-synth-data-privacy/data/duckdb/limebit_mtgan_preprocessed.duckdb")
if db.is_file():
    print('Connecting...')
    conn = duckdb.connect(database=db) 
    
# Retrieve the list of tables in the current schema
tables = conn.execute("SHOW TABLES").fetchall()
print("\nList of tables:")
for table in tables:
    # Each row is a tuple; the table name is typically the first element
    table_name = table[0]
    row_count = conn.execute(f"SELECT COUNT(*) FROM {table_name}").fetchone()[0]
    print(f"{table_name} ({row_count} rows)")

Connecting...

List of tables:
drugs (347228 rows)
inpatient_cases (14992 rows)
inpatient_diagnosis (297452 rows)
inpatient_fees (7408 rows)
inpatient_procedures (32522 rows)
insurance_data (22600 rows)
insurants (6743 rows)
outpatient_cases (135579 rows)
outpatient_diagnosis (2272498 rows)
outpatient_fees (348228 rows)
outpatient_procedures (12618 rows)


In [117]:
# Query to join insurants with insurance data and filter for 2014 end dates
# Create a new table from the join query
query = """
CREATE OR REPLACE TABLE join_2014 AS
SELECT *
FROM
    insurants 
JOIN
    insurance_data USING (pid)
WHERE
    EXTRACT(YEAR FROM insurance_data_to) = 2014
ORDER BY
    pid, insurance_data_from
"""

# Execute the query to create the table
conn.execute(query)

# Verify the table was created and count the rows
count_query = "SELECT COUNT(*) FROM join_2014"
row_count = conn.execute(count_query).fetchone()[0]

print(f"Table 'join_2014' created successfully with {row_count} rows")

# Optional: Show a sample of the data
sample_query = "SELECT * FROM join_2014 LIMIT 5"
sample_df = conn.execute(sample_query).fetchdf()
display(sample_df)

# Close the connection
conn.close()

Table 'join_2014' created successfully with 4088 rows


,pid,insurants_year_of_birth,insurants_gender,insurance_data_from,insurance_data_to,insurance_data_death,insurance_data_regional_code
0,0.0,1984.0,1,2014-01-01,2014-12-31,0,5
1,7.0,1963.0,1,2014-01-01,2014-12-31,0,9
2,10.0,1938.0,1,2014-01-01,2014-12-31,0,5
3,11.0,1971.0,2,2014-01-01,2014-12-31,0,5
4,12.0,1948.0,1,2014-01-01,2014-12-31,0,5


# Big join of all inpatient tables

In [118]:
conn = duckdb.connect(database=db)

# Create comprehensive inpatient data join in a single query
create_table_query = """
CREATE OR REPLACE TABLE join_2014_inpatient AS
SELECT 
    ins.pid,
    ins.insurants_year_of_birth,
    ins.insurants_gender,
    insd.insurance_data_from,
    insd.insurance_data_to,
    insd.insurance_data_death,
    insd.insurance_data_regional_code,
    ic.inpatient_caseID,
    ic.inpatient_cases_date_of_admission,
    ic.inpatient_cases_date_of_discharge,
    ic.inpatient_cases_cause_of_admission,
    ic.inpatient_cases_cause_of_discharge,
    ic.inpatient_cases_outpatient_treatment,
    ic.inpatient_cases_department_admission,
    ic.inpatient_cases_department_discharge,
    id.inpatient_diagnosis_diagnosis,
    id.inpatient_diagnosis_type_of_diagnosis,
    id.inpatient_diagnosis_is_main_diagnosis,
    id.inpatient_diagnosis_localisation,
    ip.inpatient_procedures_procedure_code,
    ip.inpatient_procedures_localisation,
    ip.inpatient_procedures_date_of_procedure,
    ifees.inpatient_fees_billing_code,
    ifees.inpatient_fees_amount_due,
    ifees.inpatient_fees_quantity,
    ifees.inpatient_fees_from,
    ifees.inpatient_fees_to
FROM 
    insurants ins
JOIN 
    insurance_data insd ON ins.pid = insd.pid
    AND EXTRACT(YEAR FROM insd.insurance_data_from) <= 2014 
    AND (insd.insurance_data_to IS NULL OR EXTRACT(YEAR FROM insd.insurance_data_to) >= 2014)
LEFT JOIN 
    inpatient_cases ic ON ins.pid = ic.pid
    AND (
        ic.inpatient_cases_date_of_discharge IS NULL
        OR ic.inpatient_cases_date_of_discharge BETWEEN insd.insurance_data_from AND insd.insurance_data_to
    )
LEFT JOIN 
    inpatient_diagnosis id ON ic.pid = id.pid AND ic.inpatient_caseID = id.inpatient_caseID
LEFT JOIN 
    inpatient_procedures ip ON ic.pid = ip.pid AND ic.inpatient_caseID = ip.inpatient_caseID
    AND ip.inpatient_procedures_date_of_procedure BETWEEN insd.insurance_data_from AND insd.insurance_data_to
LEFT JOIN 
    inpatient_fees ifees ON ic.pid = ifees.pid AND ic.inpatient_caseID = ifees.inpatient_caseID
    AND ifees.inpatient_fees_to BETWEEN insd.insurance_data_from AND insd.insurance_data_to
ORDER BY 
    ins.pid, 
    ic.inpatient_caseID, 
    id.inpatient_diagnosis_is_main_diagnosis DESC,
    ip.inpatient_procedures_date_of_procedure,
    ifees.inpatient_fees_from
"""

# Execute the query to create the table
conn.execute(create_table_query)

# Get comprehensive statistics about the join
statistics_query = """
SELECT
    COUNT(*) as total_rows,
    COUNT(DISTINCT pid) as unique_patients,
    COUNT(DISTINCT inpatient_caseID) as unique_cases,
    SUM(CASE WHEN inpatient_caseID IS NOT NULL THEN 1 ELSE 0 END) as rows_with_cases,
    SUM(CASE WHEN inpatient_diagnosis_diagnosis IS NOT NULL THEN 1 ELSE 0 END) as rows_with_diagnoses,
    SUM(CASE WHEN inpatient_procedures_procedure_code IS NOT NULL THEN 1 ELSE 0 END) as rows_with_procedures,
    SUM(CASE WHEN inpatient_fees_billing_code IS NOT NULL THEN 1 ELSE 0 END) as rows_with_fees,
    COUNT(DISTINCT CASE WHEN inpatient_caseID IS NOT NULL THEN pid END) as patients_with_cases,
    COUNT(DISTINCT CASE WHEN inpatient_diagnosis_diagnosis IS NOT NULL THEN pid END) as patients_with_diagnoses,
    COUNT(DISTINCT CASE WHEN inpatient_procedures_procedure_code IS NOT NULL THEN pid END) as patients_with_procedures,
    COUNT(DISTINCT CASE WHEN inpatient_fees_billing_code IS NOT NULL THEN pid END) as patients_with_fees
FROM
    join_2014_inpatient
"""
stats = conn.execute(statistics_query).fetchdf()
display(stats)

# Show a sample of the data
sample_query = """
SELECT * FROM join_2014_inpatient
LIMIT 10
"""
sample_df = conn.execute(sample_query).fetchdf()
display(sample_df)

# Close the connection
conn.close()

,total_rows,unique_patients,unique_cases,rows_with_cases,rows_with_diagnoses,rows_with_procedures,rows_with_fees,patients_with_cases,patients_with_diagnoses,patients_with_procedures,patients_with_fees
0,3652274,4088,3388,3650636.0,3650636.0,3641075.0,3546216.0,2450,2450,2102,1237


,pid,insurants_year_of_birth,insurants_gender,insurance_data_from,insurance_data_to,insurance_data_death,insurance_data_regional_code,inpatient_caseID,inpatient_cases_date_of_admission,inpatient_cases_date_of_discharge,...,inpatient_diagnosis_is_main_diagnosis,inpatient_diagnosis_localisation,inpatient_procedures_procedure_code,inpatient_procedures_localisation,inpatient_procedures_date_of_procedure,inpatient_fees_billing_code,inpatient_fees_amount_due,inpatient_fees_quantity,inpatient_fees_from,inpatient_fees_to
0,0.0,1984.0,1,2014-01-01,2014-12-31,0,5,NaN,NaT,NaT,...,NaN,NaN,None,NaN,NaT,None,NaN,NaN,NaT,NaT
1,7.0,1963.0,1,2014-01-01,2014-12-31,0,9,58.0,2014-07-22,2014-07-22,...,0.0,0.0,1-632.0,9.0,2014-07-22,47100020,16.37,1.0,2014-07-22,2014-07-22
2,7.0,1963.0,1,2014-01-01,2014-12-31,0,9,58.0,2014-07-22,2014-07-22,...,0.0,0.0,1-632.0,9.0,2014-07-22,47100020,16.37,1.0,2014-07-22,2014-07-22
3,7.0,1963.0,1,2014-01-01,2014-12-31,0,9,58.0,2014-07-22,2014-07-22,...,0.0,0.0,1-632.0,9.0,2014-07-22,47100020,16.37,1.0,2014-07-22,2014-07-22
4,7.0,1963.0,1,2014-01-01,2014-12-31,0,9,58.0,2014-07-22,2014-07-22,...,0.0,0.0,1-632.0,9.0,2014-07-22,47100020,16.37,1.0,2014-07-22,2014-07-22
5,7.0,1963.0,1,2014-01-01,2014-12-31,0,9,58.0,2014-07-22,2014-07-22,...,0.0,0.0,1-632.0,9.0,2014-07-22,47100020,16.37,1.0,2014-07-22,2014-07-22
6,7.0,1963.0,1,2014-01-01,2014-12-31,0,9,58.0,2014-07-22,2014-07-22,...,0.0,0.0,1-632.0,9.0,2014-07-22,47100020,16.37,1.0,2014-07-22,2014-07-22
7,7.0,1963.0,1,2014-01-01,2014-12-31,0,9,58.0,2014-07-22,2014-07-22,...,0.0,0.0,1-632.0,9.0,2014-07-22,47100020,16.37,1.0,2014-07-22,2014-07-22
8,7.0,1963.0,1,2014-01-01,2014-12-31,0,9,58.0,2014-07-22,2014-07-22,...,0.0,0.0,1-632.0,9.0,2014-07-22,47100020,16.37,1.0,2014-07-22,2014-07-22
9,7.0,1963.0,1,2014-01-01,2014-12-31,0,9,58.0,2014-07-22,2014-07-22,...,0.0,0.0,1-632.0,9.0,2014-07-22,47100020,16.37,1.0,2014-07-22,2014-07-22


# Big join of all outpatient tables

In [119]:
conn = duckdb.connect(database=db)

# Create comprehensive outpatient data join with 2014 filter
create_table_query = """
CREATE OR REPLACE TABLE join_2014_outpatient AS
SELECT 
    ins.pid,
    ins.insurants_year_of_birth,
    ins.insurants_gender,
    insd.insurance_data_from,
    insd.insurance_data_to,
    insd.insurance_data_death,
    insd.insurance_data_regional_code,
    oc.outpatient_caseID,
    oc.outpatient_cases_practice_code,
    oc.outpatient_cases_from,
    oc.outpatient_cases_to,
    oc.outpatient_cases_amount_due,
    oc.outpatient_cases_year,
    oc.outpatient_cases_quarter,
    od.outpatient_diagnosis_diagnosis,
    od.outpatient_diagnosis_qualification,
    od.outpatient_diagnosis_localisation,
    op.outpatient_procedures_procedure_code,
    op.outpatient_procedures_localisation,
    op.outpatient_procedures_date_of_procedure,
    op.outpatient_procedures_specialty_code,
    op.outpatient_procedures_physician_code,
    ofees.outpatient_fees_physician_code,
    ofees.outpatient_fees_specialty_code,
    ofees.outpatient_fees_billing_code,
    ofees.outpatient_fees_quantity,
    ofees.outpatient_fees_date
FROM 
    insurants ins
JOIN 
    insurance_data insd ON ins.pid = insd.pid
    AND EXTRACT(YEAR FROM insd.insurance_data_from) <= 2014 
    AND (insd.insurance_data_to IS NULL OR EXTRACT(YEAR FROM insd.insurance_data_to) >= 2014)
LEFT JOIN 
    outpatient_cases oc ON ins.pid = oc.pid
    AND oc.outpatient_cases_year = 2014
LEFT JOIN 
    outpatient_diagnosis od ON oc.pid = od.pid AND oc.outpatient_caseID = od.outpatient_caseID
LEFT JOIN 
    outpatient_procedures op ON oc.pid = op.pid AND oc.outpatient_caseID = op.outpatient_caseID
    AND op.outpatient_procedures_date_of_procedure BETWEEN insd.insurance_data_from AND insd.insurance_data_to
LEFT JOIN 
    outpatient_fees ofees ON oc.pid = ofees.pid AND oc.outpatient_caseID = ofees.outpatient_caseID
    AND ofees.outpatient_fees_date BETWEEN insd.insurance_data_from AND insd.insurance_data_to
ORDER BY 
    ins.pid, 
    oc.outpatient_caseID, 
    oc.outpatient_cases_from,
    op.outpatient_procedures_date_of_procedure,
    ofees.outpatient_fees_date
"""

# Execute the query to create the table
conn.execute(create_table_query)

# Get comprehensive statistics about the join
statistics_query = """
SELECT
    COUNT(*) as total_rows,
    COUNT(DISTINCT pid) as unique_patients,
    COUNT(DISTINCT outpatient_caseID) as unique_cases,
    SUM(CASE WHEN outpatient_caseID IS NOT NULL THEN 1 ELSE 0 END) as rows_with_cases,
    SUM(CASE WHEN outpatient_diagnosis_diagnosis IS NOT NULL THEN 1 ELSE 0 END) as rows_with_diagnoses,
    SUM(CASE WHEN outpatient_procedures_procedure_code IS NOT NULL THEN 1 ELSE 0 END) as rows_with_procedures,
    SUM(CASE WHEN outpatient_fees_billing_code IS NOT NULL THEN 1 ELSE 0 END) as rows_with_fees,
    COUNT(DISTINCT CASE WHEN outpatient_caseID IS NOT NULL THEN pid END) as patients_with_cases,
    COUNT(DISTINCT CASE WHEN outpatient_diagnosis_diagnosis IS NOT NULL THEN pid END) as patients_with_diagnoses,
    COUNT(DISTINCT CASE WHEN outpatient_procedures_procedure_code IS NOT NULL THEN pid END) as patients_with_procedures,
    COUNT(DISTINCT CASE WHEN outpatient_fees_billing_code IS NOT NULL THEN pid END) as patients_with_fees
FROM
    join_2014_outpatient
"""
stats = conn.execute(statistics_query).fetchdf()
display(stats)

# Show a sample of the data
sample_query = """
SELECT * FROM join_2014_outpatient
LIMIT 10
"""
sample_df = conn.execute(sample_query).fetchdf()
display(sample_df)

# Close the connection
conn.close()

,total_rows,unique_patients,unique_cases,rows_with_cases,rows_with_diagnoses,rows_with_procedures,rows_with_fees,patients_with_cases,patients_with_diagnoses,patients_with_procedures,patients_with_fees
0,10278033,4088,32651,10278000.0,10278000.0,9299419.0,10228356.0,4055,4055,2380,3957


,pid,insurants_year_of_birth,insurants_gender,insurance_data_from,insurance_data_to,insurance_data_death,insurance_data_regional_code,outpatient_caseID,outpatient_cases_practice_code,outpatient_cases_from,...,outpatient_procedures_procedure_code,outpatient_procedures_localisation,outpatient_procedures_date_of_procedure,outpatient_procedures_specialty_code,outpatient_procedures_physician_code,outpatient_fees_physician_code,outpatient_fees_specialty_code,outpatient_fees_billing_code,outpatient_fees_quantity,outpatient_fees_date
0,0.0,1984.0,1,2014-01-01,2014-12-31,0,5,0.0,355673673,2014-07-07,...,None,NaN,NaT,None,None,338697415,50,13610,1.0,2014-08-02
1,0.0,1984.0,1,2014-01-01,2014-12-31,0,5,0.0,355673673,2014-07-07,...,None,NaN,NaT,None,None,338697415,50,13610,1.0,2014-08-02
2,0.0,1984.0,1,2014-01-01,2014-12-31,0,5,0.0,355673673,2014-07-07,...,None,NaN,NaT,None,None,338697415,50,13610,1.0,2014-08-02
3,0.0,1984.0,1,2014-01-01,2014-12-31,0,5,0.0,355673673,2014-07-07,...,None,NaN,NaT,None,None,338697415,50,13610,1.0,2014-08-02
4,0.0,1984.0,1,2014-01-01,2014-12-31,0,5,0.0,355673673,2014-07-07,...,None,NaN,NaT,None,None,338697415,50,13610,1.0,2014-08-02
5,0.0,1984.0,1,2014-01-01,2014-12-31,0,5,0.0,355673673,2014-07-07,...,None,NaN,NaT,None,None,338697415,50,13610,1.0,2014-08-02
6,0.0,1984.0,1,2014-01-01,2014-12-31,0,5,0.0,355673673,2014-07-07,...,None,NaN,NaT,None,None,338697415,50,13610,1.0,2014-08-02
7,0.0,1984.0,1,2014-01-01,2014-12-31,0,5,0.0,355673673,2014-07-07,...,None,NaN,NaT,None,None,338697415,50,13610,1.0,2014-08-02
8,0.0,1984.0,1,2014-01-01,2014-12-31,0,5,0.0,355673673,2014-07-07,...,None,NaN,NaT,None,None,338697415,50,13610,1.0,2014-08-02
9,0.0,1984.0,1,2014-01-01,2014-12-31,0,5,0.0,355673673,2014-07-07,...,None,NaN,NaT,None,None,338697415,50,13610,1.0,2014-08-02


# Join drugs

In [120]:
conn = duckdb.connect(database=db)

# Create drug data join with 2014 filter
create_table_query = """
CREATE OR REPLACE TABLE join_2014_drugs AS
SELECT 
    ins.pid,
    ins.insurants_year_of_birth,
    ins.insurants_gender,
    insd.insurance_data_from,
    insd.insurance_data_to,
    insd.insurance_data_death,
    insd.insurance_data_regional_code,
    d.drugs_date_of_prescription,
    d.drugs_date_of_dispense,
    d.drugs_pharma_central_number,
    d.drugs_specialty_of_prescriber,
    d.drugs_physician_code,
    d.drugs_practice_code,
    d.drugs_quantity,
    d.drugs_amount_due,
    d.drugs_atc,
    d.drugs_ddd
FROM 
    insurants ins
JOIN 
    insurance_data insd ON ins.pid = insd.pid
    AND EXTRACT(YEAR FROM insd.insurance_data_from) <= 2014 
    AND (insd.insurance_data_to IS NULL OR EXTRACT(YEAR FROM insd.insurance_data_to) >= 2014)
LEFT JOIN 
    drugs d ON ins.pid = d.pid
    AND d.drugs_date_of_dispense BETWEEN insd.insurance_data_from AND insd.insurance_data_to
ORDER BY 
    ins.pid, 
    d.drugs_date_of_dispense
"""

conn.execute(create_table_query)

# Get comprehensive statistics about the drugs join
statistics_query = """
SELECT
    COUNT(*) as total_rows,
    COUNT(DISTINCT pid) as unique_patients,
    SUM(CASE WHEN drugs_pharma_central_number IS NOT NULL THEN 1 ELSE 0 END) as rows_with_drugs,
    COUNT(DISTINCT CASE WHEN drugs_pharma_central_number IS NOT NULL THEN pid END) as patients_with_drugs,
    COUNT(DISTINCT drugs_pharma_central_number) as unique_drug_codes,
    COUNT(DISTINCT drugs_atc) as unique_atc_codes,
    AVG(drugs_quantity) as avg_drug_quantity,
    AVG(drugs_amount_due) as avg_drug_cost,
    MIN(drugs_date_of_dispense) as earliest_dispense_date,
    MAX(drugs_date_of_dispense) as latest_dispense_date,
    COUNT(DISTINCT drugs_physician_code) as unique_prescribers,
    COUNT(DISTINCT drugs_practice_code) as unique_practices
FROM
    join_2014_drugs
"""

conn = duckdb.connect(database=db)
stats = conn.execute(statistics_query).fetchdf()
display(stats)

# Show a sample of the data
sample_query = """
SELECT * FROM join_2014_drugs
LIMIT 10
"""
sample_df = conn.execute(sample_query).fetchdf()
display(sample_df)

conn.close()

,total_rows,unique_patients,rows_with_drugs,patients_with_drugs,unique_drug_codes,unique_atc_codes,avg_drug_quantity,avg_drug_cost,earliest_dispense_date,latest_dispense_date,unique_prescribers,unique_practices
0,91578,4088,91577.0,4087,6925,1071,1.042733,162.741919,2014-01-01,2014-12-31,10337,7982


,pid,insurants_year_of_birth,insurants_gender,insurance_data_from,insurance_data_to,insurance_data_death,insurance_data_regional_code,drugs_date_of_prescription,drugs_date_of_dispense,drugs_pharma_central_number,drugs_specialty_of_prescriber,drugs_physician_code,drugs_practice_code,drugs_quantity,drugs_amount_due,drugs_atc,drugs_ddd
0,0.0,1984.0,1,2014-01-01,2014-12-31,0,5,2014-07-07,2014-07-07,07701898,01,963817501,985740583,1.0,26.86,J01DC02,20.000
1,0.0,1984.0,1,2014-01-01,2014-12-31,0,5,2014-07-29,2014-08-01,09372878,01,364542201,935861467,1.0,17.23,UNKNOWN,0.000
2,0.0,1984.0,1,2014-01-01,2014-12-31,0,5,2014-08-01,2014-08-01,03158931,05,286546905,056176562,1.0,0.00,B05BA11,20.000
3,0.0,1984.0,1,2014-01-01,2014-12-31,0,5,2014-08-01,2014-08-01,13245654,05,608665905,409952823,1.0,41.52,S01ED66,50.000
4,0.0,1984.0,1,2014-01-01,2014-12-31,0,5,2014-07-30,2014-08-01,05380527,01,479612701,292776267,1.0,54.83,C01BD01,100.000
5,0.0,1984.0,1,2014-01-01,2014-12-31,0,5,2014-08-17,2014-08-17,00811804,01,635825301,525742464,1.0,16.27,H03AA01,100.000
6,0.0,1984.0,1,2014-01-01,2014-12-31,0,5,2014-08-17,2014-08-17,09670009,01,771683201,098469950,1.0,16.19,C10AA05,100.000
7,0.0,1984.0,1,2014-01-01,2014-12-31,0,5,2014-09-24,2014-09-24,01038950,51,549459451,641197849,1.0,42.72,N02AX51,25.000
8,0.0,1984.0,1,2014-01-01,2014-12-31,0,5,2014-09-24,2014-09-24,08668358,31,873979931,782611280,1.0,16.33,N05CM22,33.333
9,0.0,1984.0,1,2014-01-01,2014-12-31,0,5,2014-09-24,2014-09-24,01401267,51,062783251,175771465,1.0,18.57,N03AG01,20.000


In [106]:
conn.close()